# Examen ETL: SPARK 12+1/02

* Autor: Beltrán Aller López
* Fecha: 02/12/2019
* Asignatura: ETL

Se podrá utilizar toda la información que se encuentra en el campus. 

Se va a trabajar sobre varios ficheros de datos:

Usuarios: id_usuario::sexo::edad::id_profesion::codigo_postal

Peliculas: id_pelicula::titulo (año)::tipo1|tipo2|tipo3

Ratings: id_pelicula::id_usuario::puntuacion::fecha_timestamp

A cada una de las preguntas hay que responder explicando brevemente que se pretende hacer antes de lanzar el código.

Al documento lo llamareís con vuestro nombre y apellido. Debeís enviarlo a mi correo de CUNEF antes del final del examen.

El lenguaje para trabajar con Spark podrá ser python o R indistintamente.

In [1]:
# Nota: Carga de las puntuaciones
# Función para parsear la fecha

from datetime import datetime
dateparse = lambda x: datetime.fromtimestamp(float(x))

## Primera tarea: Inicializar spark context y cargar los datos desde los ficheros.

En primer lugar inicializo el contexto spark, para ello importo la librería necesaria.

In [2]:
from pyspark import SparkContext # importo la libreria
sc = SparkContext() # inicializo el contexto spark

A continuación procedo a cargar los ficheros y verificar que la carga se realiza de forma adecuada.

### Movies 

El primer fichero que cargaré será movies.dat.txt.

In [3]:
movies_file = "../data/movies.dat.txt" 
movies_data = sc.textFile(movies_file)

* Extraigo una muestra para ver que formato tiene.

In [4]:
movies_data.take(3)

["1::Toy Story (1995)::Animation|Children's|Comedy",
 "2::Jumanji (1995)::Adventure|Children's|Fantasy",
 '3::Grumpier Old Men (1995)::Comedy|Romance']

* Realizo un conteo para comprobar que se ha cargado por completo.

In [5]:
movies_data.count()

3883

El fichero presenta las siguientes características:
* Posee 3883 registros.
* Cada registro es de la forma: "1 ó + dígitos :: cadena de texto (separada por espacios en blanco) y (4 dígitos) :: Cadena de texto separada por |".

Dejo el fichero ya dividido por los ::

In [6]:
movies_splitted = movies_data.map(lambda x: x.split("::"))

In [7]:
movies_splitted.take(1)

[['1', 'Toy Story (1995)', "Animation|Children's|Comedy"]]

### Ratings 

El siguiente fichero será ratings.dat.txt.

In [8]:
ratings_file = "../data/ratings.dat.txt" 
ratings_data = sc.textFile(ratings_file)

* Observo una muestra para ver el formato.

In [9]:
ratings_data.take(3)

['1::1193::5::978300760', '1::661::3::978302109', '1::914::3::978301968']

* Cuento el número de registros del fichero.

In [10]:
ratings_data.count()

1000209

El fichero presenta las siguientes características:
* Posee 1.000.209 registros.
* Cada registro es de la forma: '1 o + dígitos :: 4 dígitos :: dígito :: 9 dígitos'.

Dejo el fichero ya dividido por los ::

In [11]:
ratings_splitted = ratings_data.map(lambda x: x.split("::")).map(lambda x: [int(x[0]),int(x[1]),int(x[2]),datetime.fromtimestamp(float(x[3])).year])

In [12]:
ratings_splitted.take(1)

[[1, 1193, 5, 2000]]

### Users

El último fichero es users.dat.txt.

In [13]:
users_file = "../data/users.dat.txt" 
users_data = sc.textFile(users_file)

* Observo una muestra.

In [14]:
users_data.take(3)

['1::F::1::10::48067', '2::M::56::16::70072', '3::M::25::15::55117']

* Realizo el conteo del número de registros.

In [15]:
users_data.count()

6040

El fichero presenta las siguientes características:
* Posee un total de 6040 registros.
* Tiene un formato:'1 ó + dígitos :: caracter :: 1 ó + dígitos :: 2 dígitos :: 5 dígitos'

Dejo el fichero ya dividido por los ::

In [16]:
users_splitted = users_data.map(lambda x: x.split("::"))

In [17]:
users_splitted.take(1)

[['1', 'F', '1', '10', '48067']]

## Segunda tarea: Media de puntuaciones globales por año. ¿Hay algún año significativamente distinto?

In [18]:
ratings_splitted.map(lambda x: x[3]).distinct().collect()

[2000, 2001, 2002, 2003]

#### Año 2000

Saco la puntuación de cada peli del año 2000.

In [19]:
anno_2000 = ratings_splitted.filter(lambda x: x[3] == 2000).map(lambda x: (x[0],x[2])).reduceByKey(lambda a,b:a + b)

In [20]:
anno_2000.collect()

[(1, 167),
 (2, 479),
 (3, 199),
 (4, 88),
 (5, 623),
 (6, 277),
 (7, 134),
 (8, 540),
 (9, 396),
 (10, 1216),
 (11, 208),
 (12, 88),
 (13, 366),
 (14, 83),
 (15, 663),
 (16, 106),
 (17, 860),
 (18, 1113),
 (19, 365),
 (20, 57),
 (21, 64),
 (22, 750),
 (23, 18),
 (24, 516),
 (25, 290),
 (26, 1173),
 (27, 292),
 (28, 348),
 (29, 387),
 (30, 150),
 (31, 444),
 (32, 174),
 (33, 709),
 (34, 634),
 (35, 701),
 (36, 1164),
 (37, 196),
 (38, 358),
 (39, 221),
 (40, 331),
 (41, 87),
 (42, 864),
 (43, 99),
 (44, 458),
 (45, 875),
 (46, 173),
 (47, 86),
 (48, 1832),
 (49, 357),
 (50, 132),
 (51, 153),
 (52, 281),
 (53, 2898),
 (54, 161),
 (55, 103),
 (56, 266),
 (57, 186),
 (58, 1736),
 (59, 329),
 (60, 239),
 (61, 100),
 (62, 1776),
 (63, 277),
 (64, 112),
 (65, 163),
 (66, 100),
 (67, 275),
 (68, 155),
 (69, 270),
 (70, 200),
 (71, 109),
 (72, 159),
 (73, 682),
 (74, 20),
 (75, 583),
 (76, 284),
 (77, 115),
 (78, 161),
 (79, 113),
 (80, 182),
 (81, 380),
 (82, 456),
 (83, 354),
 (84, 119),
 (8

Tengo que saber el numero de veces que votaron esa peli.

In [21]:
num_votos_peli = ratings_splitted.filter(lambda x: x[3] == 2000).map(lambda x: (x[0],1)).reduceByKey(lambda a,b:a + b)

In [22]:
num_votos_peli.collect()

[(1, 40),
 (2, 129),
 (3, 51),
 (4, 21),
 (5, 198),
 (6, 71),
 (7, 31),
 (8, 139),
 (9, 106),
 (10, 299),
 (11, 55),
 (12, 23),
 (13, 108),
 (14, 25),
 (15, 200),
 (16, 35),
 (17, 211),
 (18, 305),
 (19, 97),
 (20, 13),
 (21, 22),
 (22, 247),
 (23, 6),
 (24, 131),
 (25, 76),
 (26, 397),
 (27, 70),
 (28, 91),
 (29, 108),
 (30, 43),
 (31, 119),
 (32, 48),
 (33, 196),
 (34, 164),
 (35, 198),
 (36, 277),
 (37, 53),
 (38, 100),
 (39, 62),
 (40, 96),
 (41, 25),
 (42, 231),
 (43, 24),
 (44, 117),
 (45, 297),
 (46, 41),
 (47, 22),
 (48, 597),
 (49, 97),
 (50, 43),
 (51, 40),
 (52, 79),
 (53, 684),
 (54, 40),
 (55, 25),
 (56, 67),
 (57, 64),
 (58, 437),
 (59, 93),
 (60, 70),
 (61, 36),
 (62, 498),
 (63, 84),
 (64, 27),
 (65, 36),
 (66, 26),
 (67, 64),
 (68, 42),
 (69, 65),
 (70, 54),
 (71, 29),
 (72, 43),
 (73, 202),
 (74, 5),
 (75, 147),
 (76, 64),
 (77, 39),
 (78, 44),
 (79, 31),
 (80, 47),
 (81, 86),
 (82, 113),
 (83, 99),
 (84, 31),
 (85, 20),
 (86, 29),
 (87, 59),
 (88, 68),
 (89, 10),
 (9

Hago la media de cada peli.

In [23]:
media_peli = anno_2000.join(num_votos_peli).map(lambda x:(x[0],(x[1][0]/x[1][1]))).sortByKey()

In [24]:
media_peli.collect()

[(1, 4.175),
 (2, 3.7131782945736433),
 (3, 3.9019607843137254),
 (4, 4.190476190476191),
 (5, 3.1464646464646466),
 (6, 3.9014084507042255),
 (7, 4.32258064516129),
 (8, 3.884892086330935),
 (9, 3.7358490566037736),
 (10, 4.066889632107023),
 (11, 3.7818181818181817),
 (12, 3.8260869565217392),
 (13, 3.388888888888889),
 (14, 3.32),
 (15, 3.315),
 (16, 3.0285714285714285),
 (17, 4.075829383886256),
 (18, 3.6491803278688524),
 (19, 3.7628865979381443),
 (20, 4.384615384615385),
 (21, 2.909090909090909),
 (22, 3.0364372469635628),
 (23, 3.0),
 (24, 3.9389312977099236),
 (25, 3.8157894736842106),
 (26, 2.954659949622166),
 (27, 4.171428571428572),
 (28, 3.8241758241758244),
 (29, 3.5833333333333335),
 (30, 3.488372093023256),
 (31, 3.73109243697479),
 (32, 3.625),
 (33, 3.61734693877551),
 (34, 3.8658536585365852),
 (35, 3.54040404040404),
 (36, 4.20216606498195),
 (37, 3.69811320754717),
 (38, 3.58),
 (39, 3.564516129032258),
 (40, 3.4479166666666665),
 (41, 3.48),
 (42, 3.7402597402597

In [25]:
anno_2000.count(), media_peli.count()

(6034, 6034)

Saco la media del año 2000.

In [26]:
media_2000 = media_peli.map(lambda x: x[1]).mean()

In [27]:
media_2000

3.7084709202666426

__REPITO EL PROCESO PARA EL RESTO DE AÑOS__

#### Año 2001

In [28]:
anno_2001 = ratings_splitted.filter(lambda x: x[3] == 2001).map(lambda x: (x[0],x[2])).reduceByKey(lambda a,b:a + b)
num_votos_peli2001 = ratings_splitted.filter(lambda x: x[3] == 2001).map(lambda x: (x[0],1)).reduceByKey(lambda a,b:a + b)
media_peli2001 = anno_2001.join(num_votos_peli2001).map(lambda x:(x[0],(x[1][0]/x[1][1]))).sortByKey()
media_2001 = media_peli2001.map(lambda x: x[1]).mean()
media_2001

3.6379066147771963

#### Año 2002

In [29]:
anno_2002 = ratings_splitted.filter(lambda x: x[3] == 2002).map(lambda x: (x[0],x[2])).reduceByKey(lambda a,b:a + b)
num_votos_peli2002 = ratings_splitted.filter(lambda x: x[3] == 2002).map(lambda x: (x[0],1)).reduceByKey(lambda a,b:a + b)
media_peli2002 = anno_2002.join(num_votos_peli2002).map(lambda x:(x[0],(x[1][0]/x[1][1]))).sortByKey()
media_2002 = media_peli2002.map(lambda x: x[1]).mean()
media_2002

3.5677800178850787

#### Año 2003

In [30]:
anno_2003 = ratings_splitted.filter(lambda x: x[3] == 2003).map(lambda x: (x[0],x[2])).reduceByKey(lambda a,b:a + b)
num_votos_peli2003 = ratings_splitted.filter(lambda x: x[3] == 2003).map(lambda x: (x[0],1)).reduceByKey(lambda a,b:a + b)
media_peli2003 = anno_2003.join(num_votos_peli2003).map(lambda x:(x[0],(x[1][0]/x[1][1]))).sortByKey()
media_2003 = media_peli2003.map(lambda x: x[1]).mean()
media_2003

3.53197483211742

__SOLUCION:__ La mejor media se obtuvo en el año 2000 y no se encuentran grandes diferencias entre los años.

## Tercera pregunta: ¿Cuál es la película más votada por los mayores de 60? 

¿Cuántos usuarios distintos tengo?

In [31]:
users_splitted.map(lambda x: x[0]).distinct().count()

6040

Obtengo el id de los usuarios mayores de 60 años.

In [32]:
mayores_de_60 = users_splitted.filter(lambda x: int(x[2]) > 60).map(lambda x: x[0])

In [33]:
mayores_de_60.count()

0

¿Hay usuarios menores de 60?

In [34]:
menores_de_60 = users_splitted.filter(lambda x: int(x[2]) < 60).map(lambda x: x[0])

In [35]:
menores_de_60.count()

6040

__SOLUCION:__ No hay mayores de 60 así que no hay peli más votada.

## Cuarta pregunta: ¿Cuál es la puntuación media de las peliculas de acción del año 2000?

Obtengo las peliculas de accion del año 2000 de movies_splitted.

In [36]:
action_movies = movies_splitted.filter(lambda x: 'Action' in x[2]).filter(lambda x: '(2000)' in x[1]).map(lambda x: (int(x[0]),x[1]))

In [37]:
action_movies.take(4)

[(3300, 'Pitch Black (2000)'),
 (3316, 'Reindeer Games (2000)'),
 (3452, 'Romeo Must Die (2000)'),
 (3555, 'U-571 (2000)')]

Del ejercicio 2 ya tengo una lista con los id de las pelis y las medias del año 2000.

In [38]:
media_peli.take(4)

[(1, 4.175),
 (2, 3.7131782945736433),
 (3, 3.9019607843137254),
 (4, 4.190476190476191)]

Realizo un join.

In [39]:
media_action = action_movies.join(media_peli)

In [40]:
media_action.collect()

[(3300, ('Pitch Black (2000)', 3.9640883977900554)),
 (3316, ('Reindeer Games (2000)', 4.0)),
 (3452, ('Romeo Must Die (2000)', 4.426229508196721)),
 (3578, ('Gladiator (2000)', 3.3948339483394836)),
 (3624, ('Shanghai Noon (2000)', 4.027863777089784)),
 (3744, ('Shaft (2000)', 4.083333333333333)),
 (3898, ('Bait (2000)', 3.742574257425743)),
 (3946, ('Get Carter (2000)', 3.734848484848485)),
 (3555, ('U-571 (2000)', 3.108527131782946)),
 (3563, ('Crow: Salvation, The (2000)', 3.843601895734597)),
 (3593, ('Battlefield Earth (2000)', 3.390625)),
 (3623, ('Mission: Impossible 2 (2000)', 3.6818181818181817)),
 (3717, ('Gone in 60 Seconds (2000)', 3.78125)),
 (3753, ('Patriot, The (2000)', 3.7755102040816326)),
 (3755, ('Perfect Storm, The (2000)', 4.472222222222222)),
 (3793, ('X-Men (2000)', 2.789473684210526)),
 (3827, ('Space Cowboys (2000)', 3.747311827956989)),
 (3879, ('Art of War, The (2000)', 3.3043478260869565)),
 (3889, ('Highlander: Endgame (2000)', 4.293233082706767))]

Ya tenemos la media de cada peli de acción del 2000, también obtengo la media global de las pelis de acción del 2000.

In [60]:
medias_action_global = media_action.map(lambda x: x[1][1])

In [62]:
medias_action_global.mean()

3.766404882296022

__SOLUCION:__ La media global de las pelis de acción del 2000 es de 3.76.

## Quinta pregunta: ¿ Cuál es el año en que mayor número de usuarios votaron?

Calculo el número de usuarios distintos cada año.

#### Año 2000

In [41]:
anno2000users = ratings_splitted.filter(lambda x: x[3] == 2000).map(lambda x: x[1]).distinct().count()
anno2000users

3678

#### Año 2001

In [42]:
anno2001users = ratings_splitted.filter(lambda x: x[3] == 2001).map(lambda x: x[1]).distinct().count()
anno2001users

3289

#### Año 2002 

In [43]:
anno2002users = ratings_splitted.filter(lambda x: x[3] == 2002).map(lambda x: x[1]).distinct().count()
anno2002users

2971

#### Año 2003

In [44]:
anno2003users = ratings_splitted.filter(lambda x: x[3] == 2003).map(lambda x: x[1]).distinct().count()
anno2003users

1601

__SOLUCION:__ El año que más usuarios votaron fue en el 2000.

## Sexta pregunta: ¿ Cuál es la película con mejor puntación media?

Realiza una suma de las puntuaciones de todas las pelis.

In [45]:
pelis = ratings_splitted.map(lambda x: (x[0],x[2])).reduceByKey(lambda a,b: a + b)

In [46]:
pelis.take(5)

[(1, 222), (2, 479), (3, 199), (4, 88), (5, 623)]

Miro el número de veces ue fueron votadas.

In [47]:
votos_pelis = ratings_splitted.map(lambda x: (x[0],1)).reduceByKey(lambda a,b: a + b)

In [48]:
votos_pelis.take(5)

[(1, 53), (2, 129), (3, 51), (4, 21), (5, 198)]

Calculo la media.

In [49]:
medias = pelis.join(votos_pelis).map(lambda x:(x[0],(x[1][0]/x[1][1])))

In [51]:
medias.take(5)

[(1, 4.188679245283019),
 (2, 3.7131782945736433),
 (3, 3.9019607843137254),
 (4, 4.190476190476191),
 (5, 3.1464646464646466)]

Saco el id de la película con mayor media.

In [52]:
medias.max(lambda x: x[1])

(283, 4.962962962962963)

Selecciono la película con ese id en movies.splitted.

In [55]:
movies_splitted.filter(lambda x: int(x[0]) == 283).collect()

[['283', 'New Jersey Drive (1995)', 'Crime|Drama']]

__SOLUCIÓN:__ New Jersey Drive es la película con mejor media.